In [1]:
# Minicons Installation
# Introduction can be found https://kanishka.xyz/post/minicons-running-large-scale-behavioral-analyses-on-transformer-lms/
# Tutorial and code can be found https://github.com/kanishkamisra/minicons/blob/master/examples/surprisals.md
#!pip install minicons

from minicons import scorer
import pandas as pd
import numpy as np
import json
import csv
import re
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.formula.api as smf
import torch
from torch.utils.data import DataLoader
from transformers import GPT2LMHeadModel, GPT2TokenizerFast

/tmp/ipykernel_508860/658913012.py:7: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was too old on your system - pyarrow 10.0.1 is the current minimum supported version as of this release.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
#model_path = "gpt2-small/checkpoint-pretrainedtokenizer_10M"
#model_path = "gpt2-small/checkpoint-trainedtokenizer_10M"
#model_path = "gpt2-small/checkpoint-trainedtokenizer_10M_whitespace"
#model_path = "gpt2-small/checkpoint-pretrainedtokenizer_100M"
#model_path = "gpt2-small/checkpoint-trainedtokenizer_100M"
model_path = "gpt2-small/checkpoint-trainedtokenizer_100M_whitespace"

model = GPT2LMHeadModel.from_pretrained(model_path)
tokenizer = GPT2TokenizerFast.from_pretrained(model_path)

# wrap with minicons scorer
lm_scorer = scorer.IncrementalLMScorer(model_path, device = "cpu")

In [9]:
print("Special tokens:", tokenizer.all_special_tokens)
print("Special token IDs:", tokenizer.all_special_ids)
print("Special tokens map:", tokenizer.special_tokens_map)

Special tokens: ['<|endoftext|>']
Special token IDs: [50257]
Special tokens map: {'bos_token': '<|endoftext|>', 'eos_token': '<|endoftext|>', 'unk_token': '<|endoftext|>', 'pad_token': '<|endoftext|>'}


In [12]:
surprisals

[[('<pad>', 0.0),
  ('ĠTheĠ', 6.771759033203125),
  ('balloon', 12.995538711547852),
  ('Ġwa', 3.036536693572998),
  ('s', 0.0033515978138893843),
  ('Ġinf', 14.873947143554688),
  ('lat', 3.8473756313323975),
  ('ingĠfor', 9.053845405578613),
  ('Ġ10', 10.562509536743164),
  ('Ġminute', 3.032411813735962),
  ('s', 0.0010208890307694674)]]

In [3]:
def calculate_surprisal(sentence):
    '''
    Takes in a sentence, and outputs surprisal values for each word.
    '''
    
    input_sentence = sentence
    # token_score() function of Minicons takes in several parameters
    # if surprisal = True, the output value is surprisal instead of log likelihood
    # if base_two = True, the log likelihood will be in base 2
    # see Minicons documentations for details
    # score tokens
    token_surprisals = lm_scorer.token_score(input_sentence, surprisal = True, base_two = True)[0]
    #print(token_surprisals)

    '''
    # filter out special tokens (like <pad>)
    special_tokens = set(tokenizer.all_special_tokens + ['<pad>'])
    filtered = [
        (token, score)
        for (token, score) in token_surprisals
        if token not in special_tokens
    ]
    '''

   # expand tokens that contain multiple words
    expanded = []
    for token, score in token_surprisals:
    #for token, score in filtered:
        token = token.strip('Ġ') # remove space marker
        if token.count('Ġ') > 0:
            # multiple words inside
            words = token.split('Ġ')
            words = [word for word in words if word]  # remove empty strings
            for i, word in enumerate(words):
                expanded_token = word
                expanded.append((expanded_token, score / len(words)))  # split surprisal equally
        else:
            expanded.append((token, score))    
    #print(expanded)

    # use regex to split into words and punctuation
    words = re.findall(r'\w+|[^\w\s]', sentence)
    results = []

    token_pointer = 0

    for word in words:
        accumulated = ''
        word_surprisal = 0.0

        while token_pointer < len(expanded):
            token, surprisal = expanded[token_pointer]
            accumulated += token
            word_surprisal += surprisal
            token_pointer += 1

            if accumulated == word:
                results.append((word, word_surprisal))
                break
        else:
            results.append((word, word_surprisal))

    return results


sentence = 'I know that your friend gave a baguette to Mary last weekend.'
calculate_surprisal(sentence)

[('I', 0.0),
 ('know', 4.365269184112549),
 ('that', 3.4667656421661377),
 ('your', 9.112505912780762),
 ('friend', 6.371501922607422),
 ('gave', 10.674038887023926),
 ('a', 4.927915096282959),
 ('baguette', 38.19276142120361),
 ('to', 5.820535182952881),
 ('Mary', 13.028478622436523),
 ('last', 11.46209716796875),
 ('weekend', 7.374884128570557),
 ('.', 2.0132205486297607)]

In [4]:
def calculate_sentence_surprisal(word_surprisals):
    '''Returns total surprisal and average surprisal per word.'''
    
    scores = [score for word, score in word_surprisals]
    total = sum(scores)
    avg = total / len(scores)
    return total, avg

"""
def sum_region_surprisal(row, region_list):
    '''Sums surprisals of a specified sentence region, extracting the relevant part from sentence_surprisals using index information.'''

    sentence_surprisals = row['surprisals']
    
    # retrieve all sentence column(from 'prefix' till 'end')
    all_columns = row.index.tolist()  # get the list of column names
    prefix_index = all_columns.index('prefix')
    eos_index = all_columns.index('end')
    
    # reconstruct the full sentence from 'prefix' to 'end' (exclude NaN values)
    full_sentence = ' '.join([str(row[col]) if pd.notna(row.get(col)) else '' for col in all_columns[prefix_index:eos_index + 1]])
    
    # use regex to split the full sentence into words & punctuation
    sentence_units = re.findall(r'\w+|[^\w\s]', full_sentence)
    
    # get the start and end indices for the region_list within the full sentence
    first_region_column = region_list[0]
    first_region_start_index = all_columns.index(first_region_column)
    
    last_region_column = region_list[-1]
    last_region_end_index = all_columns.index(last_region_column)
    
    # extract the relevant slice of sentence_surprisals that corresponds to the region_list
    # calculate the starting and ending index for the region_list part
    start_index = sum([
        len(re.findall(r'\w+|[^\w\s]', str(row[col]))) for col in all_columns[prefix_index:first_region_start_index]
        if pd.notna(row[col]) 
    ])
    end_index = sum([
        len(re.findall(r'\w+|[^\w\s]', str(row[col]))) for col in all_columns[prefix_index:last_region_end_index + 1]
        if pd.notna(row[col]) 
    ])

    # extract the slice of sentence_surprisals corresponding to the region_list part
    relevant_surprisals = sentence_surprisals[start_index:end_index]
    region_surprisal = sum([score for token, score in relevant_surprisals])
    
    return region_surprisal
"""

def sum_region_surprisal(row, region_list, priority_region = None, normalize = False):
    '''Sums surprisals of a specified sentence region with optional priority region handling.'''

    sentence_surprisals = row['surprisals']
    
    # retrieve all sentence column(from 'prefix' till 'end')
    all_columns = row.index.tolist()  # get the list of column names
    prefix_index = all_columns.index('prefix')
    eos_index = all_columns.index('end')
    
    # reconstruct the full sentence from 'prefix' to 'end' (exclude NaN values)
    full_sentence = ' '.join([str(row[col]) if pd.notna(row.get(col)) else '' for col in all_columns[prefix_index:eos_index + 1]])
    
    # use regex to split the full sentence into words & punctuation
    sentence_units = re.findall(r'\w+|[^\w\s]', full_sentence)
    
    if priority_region:
        # check if priority_region has non-zero surprisal
        if pd.notna(row[priority_region]):
            # extract text for the priority_region
            priority_region_index = all_columns.index(priority_region)
            
            start_index = sum([
                len(re.findall(r'\w+|[^\w\s]', str(row[col]))) for col in all_columns[prefix_index:priority_region_index]
                if pd.notna(row[col]) 
            ])
            end_index = sum([
                len(re.findall(r'\w+|[^\w\s]', str(row[col]))) for col in all_columns[prefix_index:priority_region_index + 1]
                if pd.notna(row[col]) 
            ])
            
            # calculate the surprisal for the priority region
            relevant_surprisals = sentence_surprisals[start_index:end_index]
            priority_surprisal = sum([score for token, score in relevant_surprisals])
    
            # return priority region's surprisal score if it is non-zero
            if priority_surprisal != 0:
                return priority_surprisal
    
    # if priority region is not provided or its surprisal is zero, calculate sum of region_list surprisals

    # get the start and end indices for the region_list within the full sentence
    first_region_column = region_list[0]
    first_region_start_index = all_columns.index(first_region_column)
    
    last_region_column = region_list[-1]
    last_region_end_index = all_columns.index(last_region_column)
    
    # extract the relevant slice of sentence_surprisals that corresponds to the region_list
    # calculate the starting and ending index for the region_list part
    start_index = sum([
        len(re.findall(r'\w+|[^\w\s]', str(row[col]))) for col in all_columns[prefix_index:first_region_start_index]
        if pd.notna(row[col]) 
    ])
    end_index = sum([
        len(re.findall(r'\w+|[^\w\s]', str(row[col]))) for col in all_columns[prefix_index:last_region_end_index + 1]
        if pd.notna(row[col]) 
    ])

    # extract the slice of sentence_surprisals corresponding to the region_list part
    relevant_surprisals = sentence_surprisals[start_index:end_index]
    region_surprisal = sum([score for token, score in relevant_surprisals])
    
    if normalize == False:
        return region_surprisal
    elif normalize == True:
        return region_surprisal / len(sentence_units)

In [5]:
def split_ends(ends):
    '''Splits off sentence-final punctuation.'''
    
    match = re.match(r'^(.*?)([.!?])$', ends.strip())
    if match:
        return pd.Series([match.group(1), match.group(2)])
    else:
        return pd.Series([ends, ''])  # no end punctuation
    
def encode_wh_licensor(df):
    '''
    Adds a numeric column to the DataFrame:
    - 1 if licensor is a wh-licensor
    - -1 otherwise (licensor is "that" or absent)
    '''
    df['wh_numeric'] = df['filler'].apply(lambda x: 1 if x > 0 else -1)
    return df

def merge_sentence(row, syntactic_parts):
    '''Merges sentence columns back into full sentence.'''
    
    parts = [row[part] for part in syntactic_parts]         
    # filter out NaN or empty parts
    non_empty_parts = [str(part) for part in parts if pd.notna(part) and str(part).strip() != '']
    # join with spaces and add end punctuation
    sentence = ' '.join(non_empty_parts) + (row['end'] if pd.notna(row['end']) else '')
    return sentence

#### Statistical Analysis: Mixed-Effects Linear Regression Model

In [6]:
from IPython.display import display
import warnings
from statsmodels.tools.sm_exceptions import ConvergenceWarning

def print_summary(result):
    '''
    Prints the descriptive header from a statsmodels MixedLMResults summary,
    and displays both the fixed-effects and random-effects components as pandas DataFrames.
    '''
    with warnings.catch_warnings():
        warnings.simplefilter("ignore", ConvergenceWarning)
        
        summary_str = str(result.summary())
        lines = summary_str.split('\n')

        # PART 1: HEADER
        print("\n=== MODEL SUMMARY ===")
        for line in lines:
            if 'Coef.' in line:
                break  # stop when reaching the coefficient table
            pairs = re.findall(r'(\S[^:]*):\s+([^\s][^:]*?)(?=\s{2,}|$)', line)
            for key, value in pairs:
                print(f"{key.strip()}: {value.strip()}")

        # PART 2a: FIXED EFFECTS TABLE
        fixed_table = []
        in_fixed_table = False
        random_start_idx = None

        for i, line in enumerate(lines):
            if 'Coef.' in line:
                in_fixed_table = True
                continue
            if in_fixed_table:
                if not line.strip() or set(line.strip()) in [{'='}, {'-'}]:
                    continue  # skip empty/separator lines
                parts = line.strip().split()
                if len(parts) >= 7:
                    param = parts[0]
                    row = parts[1:7]
                    fixed_table.append([param] + row)
                else:
                    random_start_idx = i
                    break

        if fixed_table:
            df_fixed = pd.DataFrame(fixed_table, columns = [
                'Parameter', 'Coef.', 'Std.Err.', 'z', 'P>|z|', '[0.025', '0.975]'
            ])
            print("\n=== FIXED EFFECTS COEFFICIENTS ===")
            display(df_fixed)
        else:
            print("\nNo fixed-effects data found.")

        # PART 2b: RANDOM EFFECTS / VARIANCE COMPONENTS TABLE
        random_table = []
        if random_start_idx:
            for line in lines[random_start_idx:]:
                if not line.strip() or set(line.strip()) in [{'='}, {'-'}]:
                    continue
                parts = line.strip().split()
                # Look for last token ending in Var or Cov
                varcov_idx = None
                for j in reversed(range(len(parts))):
                    if parts[j].endswith('Var') or parts[j].endswith('Cov'):
                        varcov_idx = j
                        break
                if varcov_idx is not None:
                    param = ' '.join(parts[:varcov_idx + 1])
                    numeric_parts = parts[varcov_idx + 1:]
                    if len(numeric_parts) == 2:
                        val1, val2 = numeric_parts
                    elif len(numeric_parts) == 1:
                        val1, val2 = numeric_parts[0], ''
                    else:
                        val1, val2 = '', ''
                    random_table.append([param, val1, val2])

        if random_table:
            df_random = pd.DataFrame(random_table, columns = ['Parameter', 'Coef.', 'Std.Err.'])
            print("\n=== RANDOM EFFECTS / VARIANCE-COVARIANCE ===")
            display(df_random)
        else:
            print("\nNo random-effects data found.")


In [7]:
# columns: item_id (indicates sentence set), wh_licensor (0/1), gap (0/1), island_type, surprisal

warnings.simplefilter("ignore", ConvergenceWarning)

def mixed_effects_linear_regression(df, iv_list, surprisal, label):
    '''
    Fits mixed-effects model and extracts wh-licensing interaction.
    '''
    '''    
    for field in iv_list:
        if field == 'wh_numeric' or 'gap_numeric' or 'gap_distance_length':
            pass
        else:
            df[field] = df[field].astype('category')
    '''

    interaction_terms = ' * '.join(iv_list)
    random_effects = ' + '.join(iv_list)

    model = smf.mixedlm(
        f"{surprisal} ~ {interaction_terms}",
        df,
        groups = df["item_id"],
        #re_formula = f"~{random_effects}"
        re_formula = "1"
        )

    result = model.fit()
    #interaction_coef = result.params.get('wh_licensor[T.1]:gap[T.1]', None)

    print(f"\n=== {label.upper()} ===")
    #print(result.summary())
    print_summary(result)

    return result

#interaction = mixed_effects_linear_regression(df, "construction_type") # label name to be changed according to construction type

#### Gap Distance - Categorical/Continuous

In [80]:
sentence_df = pd.read_csv('test_sentences/Gap Distance.csv')
sentence_df['gap_distance'] = sentence_df['gap_distance'].fillna('null')
#sentence_df['gap_distance'].dropna(inplace = True)
#sentence_df = sentence_df[sentence_df['gap_distance'].replace(['nan', 'NaN'], np.nan).notna()]

In [84]:
sentence_df[['temp_mod', 'end']] = sentence_df['temp_mod'].apply(split_ends)
sentence_df = encode_wh_licensor(sentence_df)
sentence_df['gap_numeric'] = sentence_df['gap'].replace(0, -1)
syntactic_parts = ['prefix', 'licensor', 'subj', 'modifier', 'verb', 'object', 'prep', 'goal', 'temp_mod']
sentence_df['sentence'] = sentence_df.apply(lambda x: merge_sentence(x, syntactic_parts), axis = 1)
sentence_df['surprisals'] = sentence_df['sentence'].apply(calculate_surprisal)
sentence_df['gap_distance_length'] = sentence_df['modifier'].apply(lambda x: len(x) if pd.notna(x) else 0)
sentence_df['gap_distance'] = pd.Categorical(sentence_df['gap_distance'], categories = ['null', 'short', 'medium', 'long'], ordered = True)

Gap Position == Object

In [85]:
# modifier: short_mod, med_mod, long_mod

In [86]:
object_df = sentence_df.copy()
#object_df['region_surprisal'] = object_df.apply(lambda x: sum_region_surprisal(x, ['prep', 'goal']), axis = 1)
object_df['region_surprisal'] = object_df.apply(lambda x: sum_region_surprisal(x, ['prep', 'goal'], 'object'), axis = 1)

# filter for object gap
objectgap_df = object_df[object_df['gap_position'] == 'DO']

In [87]:
# categorical
iv_list = ['wh_numeric', 'gap_numeric', 'gap_distance']
mixed_effects_linear_regression(objectgap_df, iv_list, 'region_surprisal', 'gap distance: gap position at object')


=== GAP DISTANCE: GAP POSITION AT OBJECT ===

=== MODEL SUMMARY ===
Model: MixedLM
Dependent Variable: region_surprisal
No. Observations: 320
Method: REML
No. Groups: 20
Scale: 23.7384
Min. group size: 16
Log-Likelihood: -970.7107
Max. group size: 16
Converged: Yes
Mean group size: 16.0

=== FIXED EFFECTS COEFFICIENTS ===


,Parameter,Coef.,Std.Err.,z,P>|z|,[0.025,0.975]
0,Intercept,23.533,1.024,22.979,0.000,21.526,25.540
1,gap_distance[T.short],-0.923,0.770,-1.198,0.231,-2.432,0.587
2,gap_distance[T.medium],-0.992,0.770,-1.287,0.198,-2.502,0.518
3,gap_distance[T.long],-1.468,0.770,-1.905,0.057,-2.978,0.042
4,wh_numeric,0.675,0.545,1.240,0.215,-0.392,1.743
5,wh_numeric:gap_distance[T.short],-0.360,0.770,-0.467,0.640,-1.870,1.150
6,wh_numeric:gap_distance[T.medium],-0.500,0.770,-0.650,0.516,-2.010,1.009
7,wh_numeric:gap_distance[T.long],-0.550,0.770,-0.713,0.476,-2.059,0.960
8,gap_numeric,-2.029,0.545,-3.724,0.000,-3.096,-0.961
9,gap_numeric:gap_distance[T.short],-0.086,0.770,-0.111,0.911,-1.596,1.424



=== RANDOM EFFECTS / VARIANCE-COVARIANCE ===


,Parameter,Coef.,Std.Err.
0,Group Var,15.042,1.137


In [88]:
# continuous
iv_list = ['wh_numeric', 'gap_numeric', 'gap_distance_length']
mixed_effects_linear_regression(objectgap_df, iv_list, 'region_surprisal', 'gap distance: gap position at object')


=== GAP DISTANCE: GAP POSITION AT OBJECT ===

=== MODEL SUMMARY ===
Model: MixedLM
Dependent Variable: region_surprisal
No. Observations: 320
Method: REML
No. Groups: 20
Scale: 23.1863
Min. group size: 16
Log-Likelihood: -992.7386
Max. group size: 16
Converged: Yes
Mean group size: 16.0

=== FIXED EFFECTS COEFFICIENTS ===


,Parameter,Coef.,Std.Err.,z,P>|z|,[0.025,0.975]
0,Intercept,23.303,0.970,24.019,0.000,21.402,25.205
1,wh_numeric,0.562,0.428,1.312,0.190,-0.278,1.401
2,gap_numeric,-2.467,0.428,-5.762,0.000,-3.306,-1.628
3,wh_numeric:gap_numeric,-0.353,0.428,-0.824,0.410,-1.192,0.486
4,gap_distance_length,-0.017,0.009,-1.835,0.066,-0.036,0.001
5,wh_numeric:gap_distance_length,-0.007,0.009,-0.717,0.473,-0.025,0.012
6,gap_numeric:gap_distance_length,0.005,0.009,0.496,0.620,-0.014,0.023
7,wh_numeric:gap_numeric:gap_distance_length,0.005,0.009,0.536,0.592,-0.013,0.023



=== RANDOM EFFECTS / VARIANCE-COVARIANCE ===


,Parameter,Coef.,Std.Err.
0,Group Var,15.127,1.153


Gap Postion == PP/Goal

In [89]:
pp_df = sentence_df.copy()
#pp_df['region_surprisal'] = pp_df.apply(lambda x: sum_region_surprisal(x, ['temp_mod']), axis = 1)
pp_df['region_surprisal'] = pp_df.apply(lambda x: sum_region_surprisal(x, ['temp_mod'], 'goal'), axis = 1)

# filter for PP gap
ppgap_df = pp_df[pp_df['gap_position'] == 'IO']

In [90]:
# categorical
iv_list = ['wh_numeric', 'gap_numeric', 'gap_distance']
mixed_effects_linear_regression(ppgap_df, iv_list, 'region_surprisal', 'gap distance: gap position at pp/goal')


=== GAP DISTANCE: GAP POSITION AT PP/GOAL ===

=== MODEL SUMMARY ===
Model: MixedLM
Dependent Variable: region_surprisal
No. Observations: 320
Method: REML
No. Groups: 20
Scale: 36.0144
Min. group size: 16
Log-Likelihood: -1035.8511
Max. group size: 16
Converged: Yes
Mean group size: 16.0

=== FIXED EFFECTS COEFFICIENTS ===


,Parameter,Coef.,Std.Err.,z,P>|z|,[0.025,0.975]
0,Intercept,22.152,1.360,16.286,0.000,19.486,24.818
1,gap_distance[T.short],-0.389,0.949,-0.410,0.682,-2.249,1.471
2,gap_distance[T.medium],-0.226,0.949,-0.239,0.811,-2.086,1.633
3,gap_distance[T.long],-0.533,0.949,-0.561,0.575,-2.392,1.327
4,wh_numeric,-0.006,0.671,-0.009,0.993,-1.321,1.309
5,wh_numeric:gap_distance[T.short],0.040,0.949,0.042,0.967,-1.820,1.899
6,wh_numeric:gap_distance[T.medium],0.035,0.949,0.037,0.971,-1.825,1.895
7,wh_numeric:gap_distance[T.long],0.007,0.949,0.007,0.994,-1.853,1.866
8,gap_numeric,4.894,0.671,7.294,0.000,3.579,6.209
9,gap_numeric:gap_distance[T.short],-0.294,0.949,-0.310,0.757,-2.154,1.566



=== RANDOM EFFECTS / VARIANCE-COVARIANCE ===


,Parameter,Coef.,Std.Err.
0,Group Var,27.997,1.689


In [91]:
# continuous
iv_list = ['wh_numeric', 'gap_numeric', 'gap_distance_length']
mixed_effects_linear_regression(ppgap_df, iv_list, 'region_surprisal', 'gap distance: gap position at pp/goal')


=== GAP DISTANCE: GAP POSITION AT PP/GOAL ===

=== MODEL SUMMARY ===
Model: MixedLM
Dependent Variable: region_surprisal
No. Observations: 320
Method: REML
No. Groups: 20
Scale: 35.0812
Min. group size: 16
Log-Likelihood: -1059.1465
Max. group size: 16
Converged: Yes
Mean group size: 16.0

=== FIXED EFFECTS COEFFICIENTS ===


,Parameter,Coef.,Std.Err.,z,P>|z|,[0.025,0.975]
0,Intercept,22.044,1.299,16.972,0.000,19.499,24.590
1,wh_numeric,0.015,0.527,0.028,0.978,-1.017,1.047
2,gap_numeric,4.935,0.527,9.370,0.000,3.902,5.967
3,wh_numeric:gap_numeric,-0.169,0.527,-0.321,0.748,-1.201,0.863
4,gap_distance_length,-0.005,0.012,-0.434,0.664,-0.028,0.018
5,wh_numeric:gap_distance_length,-0.000,0.012,-0.001,0.999,-0.023,0.023
6,gap_numeric:gap_distance_length,-0.001,0.012,-0.088,0.930,-0.024,0.022
7,wh_numeric:gap_numeric:gap_distance_length,0.001,0.012,0.118,0.906,-0.021,0.024



=== RANDOM EFFECTS / VARIANCE-COVARIANCE ===


,Parameter,Coef.,Std.Err.
0,Group Var,28.144,1.715


Embedded Clause

In [92]:
embed = ['subj', 'modifier', 'verb', 'object', 'prep', 'goal', 'temp_mod', 'end']
sentence_df['embed_surprisal'] = sentence_df.apply(lambda x: sum_region_surprisal(x, embed, normalize = True), axis = 1)

Embedded Clause - Object Gap Position

In [93]:
# filter for object gap
embed_objectgap_df = sentence_df[sentence_df['gap_position'] == 'DO']

In [94]:
# categorical
iv_list = ['wh_numeric', 'gap_numeric', 'gap_distance']
mixed_effects_linear_regression(embed_objectgap_df, iv_list, 'embed_surprisal', 'gap distance: embedded clause object gap position')


=== GAP DISTANCE: EMBEDDED CLAUSE OBJECT GAP POSITION ===

=== MODEL SUMMARY ===
Model: MixedLM
Dependent Variable: embed_surprisal
No. Observations: 320
Method: REML
No. Groups: 20
Scale: 0.1944
Min. group size: 16
Log-Likelihood: -252.9629
Max. group size: 16
Converged: Yes
Mean group size: 16.0

=== FIXED EFFECTS COEFFICIENTS ===


,Parameter,Coef.,Std.Err.,z,P>|z|,[0.025,0.975]
0,Intercept,6.087,0.165,36.804,0.000,5.763,6.411
1,gap_distance[T.short],1.091,0.070,15.647,0.000,0.954,1.227
2,gap_distance[T.medium],1.131,0.070,16.221,0.000,0.994,1.267
3,gap_distance[T.long],1.937,0.070,27.782,0.000,1.800,2.073
4,wh_numeric,0.116,0.049,2.356,0.018,0.020,0.213
5,wh_numeric:gap_distance[T.short],0.028,0.070,0.407,0.684,-0.108,0.165
6,wh_numeric:gap_distance[T.medium],0.012,0.070,0.169,0.866,-0.125,0.148
7,wh_numeric:gap_distance[T.long],-0.015,0.070,-0.217,0.828,-0.152,0.122
8,gap_numeric,-0.314,0.049,-6.362,0.000,-0.410,-0.217
9,gap_numeric:gap_distance[T.short],0.161,0.070,2.311,0.021,0.025,0.298



=== RANDOM EFFECTS / VARIANCE-COVARIANCE ===


,Parameter,Coef.,Std.Err.
0,Group Var,0.498,0.388


In [95]:
# continuous
iv_list = ['wh_numeric', 'gap_numeric', 'gap_distance_length']
mixed_effects_linear_regression(embed_objectgap_df, iv_list, 'embed_surprisal', 'gap distance: embedded clause object gap position')


=== GAP DISTANCE: EMBEDDED CLAUSE OBJECT GAP POSITION ===

=== MODEL SUMMARY ===
Model: MixedLM
Dependent Variable: embed_surprisal
No. Observations: 320
Method: REML
No. Groups: 20
Scale: 0.2407
Min. group size: 16
Log-Likelihood: -290.1443
Max. group size: 16
Converged: Yes
Mean group size: 16.0

=== FIXED EFFECTS COEFFICIENTS ===


,Parameter,Coef.,Std.Err.,z,P>|z|,[0.025,0.975]
0,Intercept,6.311,0.161,39.283,0.000,5.996,6.626
1,wh_numeric,0.134,0.044,3.064,0.002,0.048,0.219
2,gap_numeric,-0.274,0.044,-6.291,0.000,-0.360,-0.189
3,wh_numeric:gap_numeric,-0.028,0.044,-0.642,0.521,-0.113,0.057
4,gap_distance_length,0.023,0.001,23.864,0.000,0.021,0.025
5,wh_numeric:gap_distance_length,-0.000,0.001,-0.333,0.739,-0.002,0.002
6,gap_numeric:gap_distance_length,0.003,0.001,3.173,0.002,0.001,0.005
7,wh_numeric:gap_numeric:gap_distance_length,0.000,0.001,0.507,0.612,-0.001,0.002



=== RANDOM EFFECTS / VARIANCE-COVARIANCE ===


,Parameter,Coef.,Std.Err.
0,Group Var,0.478,0.336


Embedded Clause - PP/Goal Gap Position

In [96]:
# filter for PP gap
embed_ppgap_df = sentence_df[sentence_df['gap_position'] == 'IO']

In [97]:
# categorical
iv_list = ['wh_numeric', 'gap_numeric', 'gap_distance']
mixed_effects_linear_regression(embed_ppgap_df, iv_list, 'embed_surprisal', 'gap distance: embedded clause pp/goal gap position')


=== GAP DISTANCE: EMBEDDED CLAUSE PP/GOAL GAP POSITION ===

=== MODEL SUMMARY ===
Model: MixedLM
Dependent Variable: embed_surprisal
No. Observations: 320
Method: REML
No. Groups: 20
Scale: 0.2022
Min. group size: 16
Log-Likelihood: -256.9229
Max. group size: 16
Converged: Yes
Mean group size: 16.0

=== FIXED EFFECTS COEFFICIENTS ===


,Parameter,Coef.,Std.Err.,z,P>|z|,[0.025,0.975]
0,Intercept,6.350,0.153,41.573,0.000,6.051,6.650
1,gap_distance[T.short],0.988,0.071,13.899,0.000,0.849,1.128
2,gap_distance[T.medium],1.035,0.071,14.553,0.000,0.895,1.174
3,gap_distance[T.long],1.802,0.071,25.340,0.000,1.662,1.941
4,wh_numeric,0.093,0.050,1.842,0.065,-0.006,0.191
5,wh_numeric:gap_distance[T.short],-0.010,0.071,-0.134,0.893,-0.149,0.130
6,wh_numeric:gap_distance[T.medium],-0.014,0.071,-0.195,0.845,-0.153,0.125
7,wh_numeric:gap_distance[T.long],-0.036,0.071,-0.504,0.614,-0.175,0.104
8,gap_numeric,0.006,0.050,0.127,0.899,-0.092,0.105
9,gap_numeric:gap_distance[T.short],0.064,0.071,0.897,0.370,-0.076,0.203



=== RANDOM EFFECTS / VARIANCE-COVARIANCE ===


,Parameter,Coef.,Std.Err.
0,Group Var,0.416,0.319


In [98]:
# continuous
iv_list = ['wh_numeric', 'gap_numeric', 'gap_distance_length']
mixed_effects_linear_regression(embed_ppgap_df, iv_list, 'embed_surprisal', 'gap distance: embedded clause pp/goal gap position')


=== GAP DISTANCE: EMBEDDED CLAUSE PP/GOAL GAP POSITION ===

=== MODEL SUMMARY ===
Model: MixedLM
Dependent Variable: embed_surprisal
No. Observations: 320
Method: REML
No. Groups: 20
Scale: 0.2352
Min. group size: 16
Log-Likelihood: -285.3230
Max. group size: 16
Converged: Yes
Mean group size: 16.0

=== FIXED EFFECTS COEFFICIENTS ===


,Parameter,Coef.,Std.Err.,z,P>|z|,[0.025,0.975]
0,Intercept,6.544,0.149,43.811,0.000,6.251,6.836
1,wh_numeric,0.093,0.043,2.164,0.030,0.009,0.178
2,gap_numeric,0.030,0.043,0.701,0.484,-0.054,0.115
3,wh_numeric:gap_numeric,-0.007,0.043,-0.157,0.875,-0.091,0.078
4,gap_distance_length,0.022,0.001,22.574,0.000,0.020,0.023
5,wh_numeric:gap_distance_length,-0.000,0.001,-0.462,0.644,-0.002,0.001
6,gap_numeric:gap_distance_length,0.001,0.001,1.038,0.299,-0.001,0.003
7,wh_numeric:gap_numeric:gap_distance_length,0.000,0.001,0.097,0.923,-0.002,0.002



=== RANDOM EFFECTS / VARIANCE-COVARIANCE ===


,Parameter,Coef.,Std.Err.
0,Group Var,0.409,0.292


#### Double Gaps

In [8]:
sentence_df = pd.read_csv('test_sentences/Double Gaps.csv')

In [20]:
sentence_df[['modifier', 'end']] = sentence_df['modifier'].apply(split_ends)
sentence_df = encode_wh_licensor(sentence_df)
sentence_df['gap_numeric'] = sentence_df['gap'].replace(0, -1)
syntactic_parts = ['prefix', 'licensor', 'subj', 'verb', 'object', 'modifier']
sentence_df['sentence'] = sentence_df.apply(lambda x: merge_sentence(x, syntactic_parts), axis = 1)
sentence_df['surprisals'] = sentence_df['sentence'].apply(calculate_surprisal)
sentence_df['gap_count'] = sentence_df['subject_gap'] + sentence_df['object_gap']

In [29]:
sentence_df['gap_count'] = pd.Categorical(sentence_df['gap_count'], categories = [1, 0, 2], ordered = True)

Post Gap Region - Modifier

In [11]:
#sentence_df['region_surprisal'] = sentence_df.apply(lambda x: sum_region_surprisal(x, ['modifier']), axis = 1)
sentence_df['region_surprisal'] = sentence_df.apply(lambda x: sum_region_surprisal(x, ['modifier'], 'object'), axis = 1)

In [30]:
iv_list = ['wh_numeric', 'gap_count']
mixed_effects_linear_regression(sentence_df, iv_list, 'region_surprisal', 'double gap modifier')


=== DOUBLE GAP MODIFIER ===

=== MODEL SUMMARY ===
Model: MixedLM
Dependent Variable: region_surprisal
No. Observations: 160
Method: REML
No. Groups: 20
Scale: 23.2179
Min. group size: 8
Log-Likelihood: -483.9993
Max. group size: 8
Converged: Yes
Mean group size: 8.0

=== FIXED EFFECTS COEFFICIENTS ===


,Parameter,Coef.,Std.Err.,z,P>|z|,[0.025,0.975]
0,Intercept,18.036,0.797,22.629,0.000,16.474,19.598
1,gap_count[T.0],-4.261,0.933,-4.567,0.000,-6.090,-2.432
2,gap_count[T.2],5.859,0.933,6.279,0.000,4.030,7.688
3,wh_numeric,-0.239,0.539,-0.444,0.657,-1.295,0.816
4,wh_numeric:gap_count[T.0],1.080,0.933,1.158,0.247,-0.749,2.909
5,wh_numeric:gap_count[T.2],0.396,0.933,0.425,0.671,-1.433,2.225



=== RANDOM EFFECTS / VARIANCE-COVARIANCE ===


,Parameter,Coef.,Std.Err.
0,Group Var,6.901,0.705


Embedded Clause

In [24]:
embed = ['subj', 'object', 'verb', 'modifier', 'end']
sentence_df['embed_surprisal'] = sentence_df.apply(lambda x: sum_region_surprisal(x, embed, normalize = True), axis = 1)

In [31]:
iv_list = ['wh_numeric', 'gap_count']
result = mixed_effects_linear_regression(sentence_df, iv_list, 'embed_surprisal', 'double gap embed')
result


=== DOUBLE GAP EMBED ===

=== MODEL SUMMARY ===
Model: MixedLM
Dependent Variable: embed_surprisal
No. Observations: 160
Method: REML
No. Groups: 20
Scale: 0.1380
Min. group size: 8
Log-Likelihood: -106.9654
Max. group size: 8
Converged: Yes
Mean group size: 8.0

=== FIXED EFFECTS COEFFICIENTS ===


,Parameter,Coef.,Std.Err.,z,P>|z|,[0.025,0.975]
0,Intercept,5.243,0.140,37.516,0.000,4.969,5.517
1,gap_count[T.0],0.051,0.072,0.708,0.479,-0.090,0.192
2,gap_count[T.2],0.077,0.072,1.068,0.286,-0.064,0.218
3,wh_numeric,0.001,0.042,0.020,0.984,-0.081,0.082
4,wh_numeric:gap_count[T.0],0.142,0.072,1.970,0.049,0.001,0.283
5,wh_numeric:gap_count[T.2],-0.017,0.072,-0.232,0.817,-0.158,0.124



=== RANDOM EFFECTS / VARIANCE-COVARIANCE ===


,Parameter,Coef.,Std.Err.
0,Group Var,0.356,0.348


#### Wh-Islands

In [113]:
sentence_df = pd.read_csv('test_sentences/Wh-Islands.csv')
sentence_df['comp_type'] = sentence_df['comp_type'].fillna('null')

In [114]:
sentence_df[['cont', 'end']] = sentence_df['cont'].apply(split_ends)
sentence_df = encode_wh_licensor(sentence_df)
sentence_df['gap_numeric'] = sentence_df['gap'].replace(0, -1)
syntactic_parts = ['prefix', 'filler_word', 'subj1', 'verb1', 'comp', 'subj2', 'embed_verb', 'obj', 'cont']
sentence_df['sentence'] = sentence_df.apply(lambda x: merge_sentence (x, syntactic_parts), axis = 1)
sentence_df['surprisals'] = sentence_df['sentence'].apply(calculate_surprisal)
sentence_df['comp_type'] = pd.Categorical(sentence_df['comp_type'], categories = ['null', 'that', 'whether'], ordered = True)

Post-Gap Region - Continuation

In [115]:
sentence_df['region_surprisal'] = sentence_df.apply(lambda x: sum_region_surprisal(x, ['cont'], 'obj'), axis = 1)

In [116]:
iv_list = ['wh_numeric', 'gap_numeric', 'comp_type'] # island_type = comp_type
mixed_effects_linear_regression(sentence_df, iv_list, 'region_surprisal', 'wh-islands continuation')


=== WH-ISLANDS CONTINUATION ===

=== MODEL SUMMARY ===
Model: MixedLM
Dependent Variable: region_surprisal
No. Observations: 240
Method: REML
No. Groups: 20
Scale: 37.6463
Min. group size: 12
Log-Likelihood: -784.7926
Max. group size: 12
Converged: Yes
Mean group size: 12.0

=== FIXED EFFECTS COEFFICIENTS ===


,Parameter,Coef.,Std.Err.,z,P>|z|,[0.025,0.975]
0,Intercept,20.893,1.342,15.572,0.000,18.263,23.522
1,comp_type[T.that],-0.135,0.970,-0.139,0.890,-2.036,1.767
2,comp_type[T.whether],0.223,0.970,0.230,0.818,-1.679,2.124
3,wh_numeric,0.004,0.686,0.005,0.996,-1.341,1.348
4,wh_numeric:comp_type[T.that],-0.006,0.970,-0.006,0.995,-1.908,1.895
5,wh_numeric:comp_type[T.whether],0.006,0.970,0.006,0.995,-1.896,1.907
6,gap_numeric,3.373,0.686,4.918,0.000,2.029,4.718
7,gap_numeric:comp_type[T.that],0.532,0.970,0.548,0.583,-1.369,2.434
8,gap_numeric:comp_type[T.whether],0.894,0.970,0.921,0.357,-1.008,2.795
9,wh_numeric:gap_numeric,-0.408,0.686,-0.595,0.552,-1.753,0.936



=== RANDOM EFFECTS / VARIANCE-COVARIANCE ===


,Parameter,Coef.,Std.Err.
0,Group Var,26.592,1.642


Embedded Clause

In [117]:
embed = ['subj1', 'verb1', 'comp', 'subj2', 'embed_verb', 'obj', 'cont', 'end']
sentence_df['embed_surprisal'] = sentence_df.apply(lambda x: sum_region_surprisal(x, embed, normalize = True), axis = 1)

In [118]:
iv_list = ['wh_numeric', 'gap_numeric', 'comp_type'] # island_type = comp_type
mixed_effects_linear_regression(sentence_df, iv_list, 'embed_surprisal', 'wh-islands embed')


=== WH-ISLANDS EMBED ===

=== MODEL SUMMARY ===
Model: MixedLM
Dependent Variable: embed_surprisal
No. Observations: 240
Method: REML
No. Groups: 20
Scale: 0.0393
Min. group size: 12
Log-Likelihood: -33.0366
Max. group size: 12
Converged: Yes
Mean group size: 12.0

=== FIXED EFFECTS COEFFICIENTS ===


,Parameter,Coef.,Std.Err.,z,P>|z|,[0.025,0.975]
0,Intercept,6.694,0.200,33.408,0.000,6.302,7.087
1,comp_type[T.that],-0.377,0.031,-12.027,0.000,-0.438,-0.316
2,comp_type[T.whether],0.162,0.031,5.155,0.000,0.100,0.223
3,wh_numeric,-0.305,0.022,-13.759,0.000,-0.348,-0.262
4,wh_numeric:comp_type[T.that],0.101,0.031,3.211,0.001,0.039,0.162
5,wh_numeric:comp_type[T.whether],-0.029,0.031,-0.933,0.351,-0.091,0.032
6,gap_numeric,-0.045,0.022,-2.027,0.043,-0.088,-0.001
7,gap_numeric:comp_type[T.that],0.007,0.031,0.235,0.814,-0.054,0.069
8,gap_numeric:comp_type[T.whether],0.056,0.031,1.783,0.075,-0.006,0.117
9,wh_numeric:gap_numeric,-0.053,0.022,-2.404,0.016,-0.097,-0.010



=== RANDOM EFFECTS / VARIANCE-COVARIANCE ===


,Parameter,Coef.,Std.Err.
0,Group Var,0.793,1.361


#### Adjunct Islands 

In [119]:
sentence_df = pd.read_csv('test_sentences/Adjunct Islands.csv')
sentence_df['adjunct_type'] = sentence_df['adjunct_type'].fillna('null')

In [120]:
sentence_df[['cont', 'end']] = sentence_df['cont'].apply(split_ends)
sentence_df['adjunct_front_trigger'] = np.nan
sentence_df['adjunct_front_trigger'] = sentence_df['adjunct_trigger'].where(sentence_df['adjunct_type'] == 'front', sentence_df['adjunct_front_trigger'])
sentence_df['adjunct_trigger'] = sentence_df['adjunct_trigger'].where(sentence_df['adjunct_type'] != 'front', np.nan)
sentence_df.rename(columns = {'adjunct_trigger': 'adjunct_back_trigger'}, inplace = True)
sentence_df = sentence_df[[col for col in sentence_df.columns if col != 'end'] + ['end']]

In [121]:
sentence_df = encode_wh_licensor(sentence_df)
sentence_df['gap_numeric'] = sentence_df['gap'].replace(0, -1)
syntactic_parts = ['prefix', 'filler_word', 'adjunct_back_trigger', 'prep', 'subj', 'mod', 'verb', 'obj', 'cont', 'adjunct_front_trigger']
sentence_df['sentence'] = sentence_df.apply(lambda x: merge_sentence(x, syntactic_parts), axis = 1)
sentence_df['surprisals'] = sentence_df['sentence'].apply(calculate_surprisal)
sentence_df['adjunct_type'] = pd.Categorical(sentence_df['adjunct_type'], categories = ['null', 'back', 'front'], ordered = True)

Post-Gap Region - Continuation

In [122]:
sentence_df.iloc[0]['sentence']

'We discovered that the intern at the new office was preparing for the presentation with extra care.'

In [123]:
sentence_df['region_surprisal'] = sentence_df.apply(lambda x: sum_region_surprisal(x, ['cont'], 'obj'), axis = 1)

In [124]:
iv_list = ['wh_numeric', 'gap_numeric', 'adjunct_type'] # island_type = adjunct_type
mixed_effects_linear_regression(sentence_df, iv_list, 'region_surprisal', 'adjunct islands continuation')


=== ADJUNCT ISLANDS CONTINUATION ===

=== MODEL SUMMARY ===
Model: MixedLM
Dependent Variable: region_surprisal
No. Observations: 240
Method: REML
No. Groups: 20
Scale: 30.5650
Min. group size: 12
Log-Likelihood: -766.1575
Max. group size: 12
Converged: Yes
Mean group size: 12.0

=== FIXED EFFECTS COEFFICIENTS ===


,Parameter,Coef.,Std.Err.,z,P>|z|,[0.025,0.975]
0,Intercept,28.702,1.524,18.829,0.000,25.714,31.689
1,adjunct_type[T.back],-0.992,0.874,-1.135,0.256,-2.705,0.721
2,adjunct_type[T.front],-0.296,0.874,-0.338,0.735,-2.009,1.417
3,wh_numeric,0.288,0.618,0.466,0.641,-0.923,1.500
4,wh_numeric:adjunct_type[T.back],-0.288,0.874,-0.330,0.742,-2.002,1.425
5,wh_numeric:adjunct_type[T.front],-0.229,0.874,-0.262,0.793,-1.942,1.484
6,gap_numeric,4.190,0.618,6.779,0.000,2.979,5.402
7,gap_numeric:adjunct_type[T.back],0.131,0.874,0.150,0.881,-1.582,1.845
8,gap_numeric:adjunct_type[T.front],0.242,0.874,0.277,0.782,-1.471,1.956
9,wh_numeric:gap_numeric,-0.158,0.618,-0.255,0.798,-1.369,1.054



=== RANDOM EFFECTS / VARIANCE-COVARIANCE ===


,Parameter,Coef.,Std.Err.
0,Group Var,38.830,2.536


Embedded Clause

In [125]:
embed = ['adjunct_back_trigger', 'prep', 'subj', 'mod', 'verb', 'obj', 'cont', 'adjunct_front_trigger', 'end']
sentence_df['embed_surprisal'] = sentence_df.apply(lambda x: sum_region_surprisal(x, embed, normalize = True), axis = 1)

In [126]:
iv_list = ['wh_numeric', 'gap_numeric', 'adjunct_type'] # island_type = adjunct_type
mixed_effects_linear_regression(sentence_df, iv_list, 'embed_surprisal', 'adjunct islands embed')


=== ADJUNCT ISLANDS EMBED ===

=== MODEL SUMMARY ===
Model: MixedLM
Dependent Variable: embed_surprisal
No. Observations: 240
Method: REML
No. Groups: 20
Scale: 0.0628
Min. group size: 12
Log-Likelihood: -81.5493
Max. group size: 12
Converged: Yes
Mean group size: 12.0

=== FIXED EFFECTS COEFFICIENTS ===


,Parameter,Coef.,Std.Err.,z,P>|z|,[0.025,0.975]
0,Intercept,6.196,0.195,31.719,0.000,5.814,6.579
1,adjunct_type[T.back],0.618,0.040,15.581,0.000,0.540,0.695
2,adjunct_type[T.front],1.073,0.040,27.061,0.000,0.995,1.150
3,wh_numeric,0.104,0.028,3.702,0.000,0.049,0.159
4,wh_numeric:adjunct_type[T.back],-0.079,0.040,-1.987,0.047,-0.156,-0.001
5,wh_numeric:adjunct_type[T.front],0.000,0.040,0.007,0.995,-0.077,0.078
6,gap_numeric,-0.208,0.028,-7.409,0.000,-0.263,-0.153
7,gap_numeric:adjunct_type[T.back],0.108,0.040,2.719,0.007,0.030,0.185
8,gap_numeric:adjunct_type[T.front],0.148,0.040,3.723,0.000,0.070,0.225
9,wh_numeric:gap_numeric,-0.010,0.028,-0.349,0.727,-0.065,0.045



=== RANDOM EFFECTS / VARIANCE-COVARIANCE ===


,Parameter,Coef.,Std.Err.
0,Group Var,0.748,1.017


#### Complex NP Islands

In [ ]:
sentence_df = pd.read_csv('test_sentences/Complex NP Islands.csv')

In [ ]:
sentence_df = encode_wh_licensor(sentence_df)
sentence_df['gap_numeric'] = sentence_df['gap'].replace(0, -1)
sentence_df['sentence'] = sentence_df.apply(merge_sentence, axis = 1)
sentence_df['surprisals'] = sentence_df['sentence'].apply(calculate_surprisal)
sentence_df

Subject Condition

In [ ]:
subject_df = sentence_df.copy()

In [ ]:
subject_df['region_surprisal'] = subject_df.apply(lambda x: sum_region_surprisal(x, ['subj_setup']), axis = 1)
subject_df

In [ ]:
# filter for subject condition
subject_df = subject_df[subject_df['subj_obj'] == 'subject']

In [ ]:
iv_list = ['wh_numeric', 'gap_numeric', 'island_type']
mixed_effects_linear_regression(subject_df, iv_list, 'region_surprisal', 'complex np islands: subject condition')

Object Condition

In [ ]:
object_df = sentence_df.copy()

In [ ]:
object_df['region_surprisal'] = object_df.apply(lambda x: sum_region_surprisal(x, ['end']), axis = 1)
object_df

In [ ]:
# filter for object gap
object_df = object_df[object_df['subj_obj'] == 'object']

In [ ]:
iv_list = ['wh_numeric', 'gap_numeric', 'island_type']
mixed_effects_linear_regression(object_df, iv_list, 'region_surprisal', 'complex np islands: object condition')

Embedded Clause

In [ ]:
embed = ['subj', 'that_rc', 'what_rc', 'rc_np', 'prep', 'prep_np', 'subj_setup', 'obj_setup', 'end']
sentence_df['embed_surprisal'] = sentence_df.apply(lambda x: sum_region_surprisal(x, embed, normalize = True), axis = 1)
sentence_df

Embedded Clause - Subject Condition

In [ ]:
# filter for subject condition
subject_df = sentence_df[sentence_df['subj_obj'] == 'subject']

In [ ]:
iv_list = ['wh_numeric', 'gap_numeric', 'island_type']
mixed_effects_linear_regression(subject_df, iv_list, 'embed_surprisal', 'complex np islands: embedded clause subject condition')

Embedded Clause - Object Condition

In [ ]:
# filter for object gap
object_df = sentence_df[sentence_df['subj_obj'] == 'object']

In [ ]:
iv_list = ['wh_numeric', 'gap_numeric', 'island_type']
mixed_effects_linear_regression(object_df, iv_list, 'embed_surprisal', 'complex np islands: embedded clause object condition')